In [1]:
import pandas as pd
import json
import os
import numpy as np
import matplotlib.pyplot as plt

%pwd


'C:\\Users\\ckent\\Desktop\\github_repo\\UT-Data-Science\\pandasHomework'

In [2]:
csv_path = os.path.join('..','pandasHomework','students_complete.csv')
csv_path02 = os.path.join('..','pandasHomework','schools_complete.csv')
students_df = pd.read_csv(csv_path)
schools_df = pd.read_csv(csv_path02)



In [3]:
students_df.head()


,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [4]:
#clean up column headers
schools02_df = schools_df.rename(columns={"School ID":"SCHOOL ID", "name":"SCHOOL","type":"SCHOOL TYPE","size":"ALL STUDENTS",
                                             "budget":"BUDGET"})
students02_df = students_df.rename(columns={"Student ID":"STUDENT ID","name":"NAME","gender":"GENDER",
                                               "grade":"GRADE","school":"SCHOOL","reading_score":"READING SCORE",
                                               "math_score":"MATH SCORE"})
schools02_df.head()



,SCHOOL ID,SCHOOL,SCHOOL TYPE,ALL STUDENTS,BUDGET
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [31]:
#CREATE DISTRICT SUMMARY TABLE

In [32]:
#find total number of schools
total_schools = len(students02_df["SCHOOL"].unique())

#find total number of students
total_students = students02_df["STUDENT ID"].count()

#calculate total budget
total_budget = schools02_df['BUDGET'].sum()

#calculate average math score
avg_math = students02_df["MATH SCORE"].mean()

#calculate average reading score
avg_read = students02_df["READING SCORE"].mean()

#% students passing math
pass_math = ((students02_df[(students02_df['MATH SCORE']>=70)].count()['MATH SCORE'])/total_students)*100

#% students passing reading
pass_read = ((students02_df[(students02_df['READING SCORE']>=70)].count()["READING SCORE"])/total_students)*100

#% overall passing 

total_pass = (avg_math + avg_read)/2

#create summary table
summary_table = pd.DataFrame({"TOTAL SCHOOLS":[total_schools],"TOTAL STUDENTS":[total_students],"TOTAL BUDGET":[total_budget],
                             "AVERAGE MATH SCORE":[avg_math],"AVERAGE READING SCORE":[avg_read],"% PASSING MATH":[pass_math],
                             "% PASSING READING":[pass_read],"% OVERALL PASSING RATE":[total_pass]})

final_summary_table_df = summary_table[["TOTAL SCHOOLS","TOTAL STUDENTS","TOTAL BUDGET","AVERAGE MATH SCORE",
                                     "AVERAGE READING SCORE","% PASSING MATH","% PASSING READING",
                                     "% OVERALL PASSING RATE"]]

#format output
final_summary_table_df["TOTAL STUDENTS"]=final_summary_table_df["TOTAL STUDENTS"].map("{:,}".format)
final_summary_table_df["TOTAL BUDGET"]=final_summary_table_df["TOTAL BUDGET"].map("${:,.2f}".format)
final_summary_table_df["AVERAGE MATH SCORE"]=final_summary_table_df["AVERAGE MATH SCORE"].map("{:,.2f}".format)
final_summary_table_df["AVERAGE READING SCORE"]=final_summary_table_df["AVERAGE READING SCORE"].map("{:,.2f}".format)
final_summary_table_df["% PASSING MATH"]=final_summary_table_df["% PASSING MATH"].apply('{:.2f}%'.format)
final_summary_table_df["% PASSING READING"]=final_summary_table_df["% PASSING READING"].apply('{:.2f}%'.format)
final_summary_table_df["% OVERALL PASSING RATE"]=final_summary_table_df["% OVERALL PASSING RATE"].apply('{:.2f}%'.format)
final_summary_table_df


,TOTAL SCHOOLS,TOTAL STUDENTS,TOTAL BUDGET,AVERAGE MATH SCORE,AVERAGE READING SCORE,% PASSING MATH,% PASSING READING,% OVERALL PASSING RATE
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,80.43%


In [ ]:
#CREATE SCHOOL SUMMARY TABLE

In [194]:
#merge tables  school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

school_merge_df = pd.merge(schools02_df,students02_df,how='outer',on='SCHOOL')

#inspect merged table

school_merge_df.head()

,SCHOOL ID,SCHOOL,SCHOOL TYPE,ALL STUDENTS,BUDGET,STUDENT ID,NAME,GENDER,GRADE,READING SCORE,MATH SCORE
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [193]:
#groupby school name
school_pop = school_merge_df['SCHOOL'].value_counts()

#budget per school
school_budget = school_merge_df.groupby(['SCHOOL']).mean()['BUDGET']

#budget per student for df
budget_per_student = school_budget / school_pop

#avg scores per school calculations
math_school = school_merge_df.groupby(['SCHOOL']).mean()['MATH SCORE']
read_school = school_merge_df.groupby(['SCHOOL']).mean()['READING SCORE']
math_per_school = school_merge_df[school_merge_df['MATH SCORE']>=70].groupby(['SCHOOL']).count()["MATH SCORE"]
reading_per_school = school_merge_df[school_merge_df['READING SCORE']>=70].groupby(['SCHOOL']).count()["READING SCORE"]

#avg scores per school for df
percent_pass_math = (math_per_school/school_pop)*100
percent_pass_reading = (reading_per_school/school_pop)*100
percent_overall_passing = ((percent_pass_math + percent_pass_reading)/2)

#create columns
columns = ['TOTAL STUDENTS','TOTAL SCHOOL BUDGET', 'PER STUDENT BUDGET', 'AVERAGE MATH SCORE',
           'AVERAGE READING SCORE', '% PASSING MATH', '% PASSING READING', '% OVERALL PASSING']

#Create dictionary
school_dict = {"SCHOOL":reading_per_school.index,'TOTAL STUDENTS': school_pop.values, 
               'TOTAL SCHOOL BUDGET': school_budget.values, "PER STUDENT BUDGET": budget_per_student.values,
               'AVERAGE MATH SCORE': math_school.values, 'AVERAGE READING SCORE': read_school.values, 
               '% PASSING MATH': percent_pass_math.values,'% PASSING READING': percent_pass_reading.values, 
               '% OVERALL PASSING': percent_overall_passing.values}

#set index
per_school_summary_df = pd.DataFrame(school_dict).set_index("SCHOOL")

#rearrange columns
per_school_summary_df = per_school_summary_df[['TOTAL STUDENTS','TOTAL SCHOOL BUDGET', 'PER STUDENT BUDGET', 'AVERAGE MATH SCORE',
                                               'AVERAGE READING SCORE', '% PASSING MATH', '% PASSING READING', '% OVERALL PASSING']]

#formatting
per_school_summary_df['PER STUDENT BUDGET']=per_school_summary_df['PER STUDENT BUDGET'].map('${:,.2f}'.format)             
per_school_summary_df['TOTAL SCHOOL BUDGET']=per_school_summary_df['TOTAL SCHOOL BUDGET'].map('${:,.2f}'.format)
per_school_summary_df['AVERAGE MATH SCORE']=per_school_summary_df['AVERAGE MATH SCORE'].apply('{:.2f}%'.format)
per_school_summary_df['AVERAGE READING SCORE']=per_school_summary_df['AVERAGE READING SCORE'].apply('{:.2f}%'.format)
per_school_summary_df['% PASSING MATH']=per_school_summary_df['% PASSING MATH'].apply('{:.2f}%'.format)
per_school_summary_df['% PASSING READING']=per_school_summary_df['% PASSING READING'].apply('{:.2f}%'.format)
per_school_summary_df['% OVERALL PASSING']=per_school_summary_df['% OVERALL PASSING'].apply('{:.2f}%'.format)

#call df
per_school_summary_df.head()

,TOTAL STUDENTS,TOTAL SCHOOL BUDGET,PER STUDENT BUDGET,AVERAGE MATH SCORE,AVERAGE READING SCORE,% PASSING MATH,% PASSING READING,% OVERALL PASSING
SCHOOL,,,,,,,,
Bailey High School,4976,"$3,124,928.00",$628.00,77.05%,81.03%,66.68%,81.93%,74.31%
Cabrera High School,4761,"$1,081,356.00",$582.00,83.06%,83.98%,94.13%,97.04%,95.59%
Figueroa High School,4635,"$1,884,411.00",$639.00,76.71%,81.16%,65.99%,80.74%,73.36%
Ford High School,3999,"$1,763,916.00",$644.00,77.10%,80.75%,68.31%,79.30%,73.80%
Griffin High School,2949,"$917,500.00",$625.00,83.35%,83.82%,93.39%,97.14%,95.27%


In [11]:
#TOP PERFORMING SCHOOLS (BY PASSING RATE)

In [192]:
best_rank_df = per_school_summary_df.sort_values(by=['% OVERALL PASSING'],ascending=False)
best_rank_df.head()

,TOTAL STUDENTS,TOTAL SCHOOL BUDGET,PER STUDENT BUDGET,AVERAGE MATH SCORE,AVERAGE READING SCORE,% PASSING MATH,% PASSING READING,% OVERALL PASSING,SPENDING RANGE (PER STUDENT)
SCHOOL,,,,,,,,,
Cabrera High School,4761,"$1,081,356.00",$582.00,83.06%,83.98%,94.13%,97.04%,95.59%,NaN
Thomas High School,1468,"$1,043,130.00",$638.00,83.42%,83.85%,93.27%,97.31%,95.29%,NaN
Griffin High School,2949,"$917,500.00",$625.00,83.35%,83.82%,93.39%,97.14%,95.27%,NaN
Pena High School,1800,"$585,858.00",$609.00,83.84%,84.04%,94.59%,95.95%,95.27%,NaN
Wilson High School,962,"$1,319,574.00",$578.00,83.27%,83.99%,93.87%,96.54%,95.20%,NaN


In [13]:
#BOTTOM PERFORMING SCHOOLS (BY PASSING RATE)

In [191]:
worst_rank_df = per_school_summary_df.sort_values(by=['% OVERALL PASSING'])
worst_rank_df.head()

,TOTAL STUDENTS,TOTAL SCHOOL BUDGET,PER STUDENT BUDGET,AVERAGE MATH SCORE,AVERAGE READING SCORE,% PASSING MATH,% PASSING READING,% OVERALL PASSING,SPENDING RANGE (PER STUDENT)
SCHOOL,,,,,,,,,
Rodriguez High School,1761,"$2,547,363.00",$637.00,76.84%,80.74%,66.37%,80.22%,73.29%,NaN
Figueroa High School,4635,"$1,884,411.00",$639.00,76.71%,81.16%,65.99%,80.74%,73.36%,NaN
Huang High School,2283,"$1,910,635.00",$655.00,76.63%,81.18%,65.68%,81.32%,73.50%,NaN
Johnson High School,1858,"$3,094,650.00",$650.00,77.07%,80.97%,66.06%,81.22%,73.64%,NaN
Ford High School,3999,"$1,763,916.00",$644.00,77.10%,80.75%,68.31%,79.30%,73.80%,NaN


In [15]:
#AVERAGE MATH SCORE BY GRADE LEVEL--------------------------------------------------------------------------------------------

In [190]:
#convert grade levels from cell value to column headers

students02_df['GRADE'] = (students02_df['GRADE'].replace({'GRADE' : {'9th': 9,'10th': 10,'11th': 11,'12th': 12}},
                                                        regex=True))

#group by to change index to school and grade, with formatted average math score reflected as a percentage

grade_level_df = (students02_df.reset_index().groupby(["SCHOOL","GRADE"], 
                                                      as_index=True)['MATH SCORE'].mean().apply('{:.2f}%'.format)).unstack()

#reorder columns
math_scores_by_grade = grade_level_df[['9th','10th','11th','12th']]
math_scores_by_grade.index.name = None
                                       
#call new df

math_scores_by_grade



GRADE,9th,10th,11th,12th
Bailey High School,77.08%,77.00%,77.52%,76.49%
Cabrera High School,83.09%,83.15%,82.77%,83.28%
Figueroa High School,76.40%,76.54%,76.88%,77.15%
Ford High School,77.36%,77.67%,76.92%,76.18%
Griffin High School,82.04%,84.23%,83.84%,83.36%
Hernandez High School,77.44%,77.34%,77.14%,77.19%
Holden High School,83.79%,83.43%,85.00%,82.86%
Huang High School,77.03%,75.91%,76.45%,77.23%
Johnson High School,77.19%,76.69%,77.49%,76.86%
Pena High School,83.63%,83.37%,84.33%,84.12%


In [17]:
#AVERAGE READING SCORE BY GRADE LEVEL-----------------------------------------------------------------------------------------

In [115]:
#group by to change index to school and grade, with formatted average reading score reflected as a percentage

grade_level_read_df = (students02_df.reset_index().groupby(["SCHOOL","GRADE"], 
                                                      as_index=True)['READING SCORE'].mean().apply('{:.2f}%'.format)).unstack()

#reorder columns
read_scores_by_grade = grade_level_df[['9th','10th','11th','12th']]
read_scores_by_grade.index.name = None
                                       
#call new df

read_scores_by_grade

GRADE,9th,10th,11th,12th
Bailey High School,77.08%,77.00%,77.52%,76.49%
Cabrera High School,83.09%,83.15%,82.77%,83.28%
Figueroa High School,76.40%,76.54%,76.88%,77.15%
Ford High School,77.36%,77.67%,76.92%,76.18%
Griffin High School,82.04%,84.23%,83.84%,83.36%
Hernandez High School,77.44%,77.34%,77.14%,77.19%
Holden High School,83.79%,83.43%,85.00%,82.86%
Huang High School,77.03%,75.91%,76.45%,77.23%
Johnson High School,77.19%,76.69%,77.49%,76.86%
Pena High School,83.63%,83.37%,84.33%,84.12%


In [19]:
#SCORES BY SCHOOL SPENDING

In [116]:
#find max & min spending for bins
print(per_school_summary_df["PER STUDENT BUDGET"].max())

print(per_school_summary_df["PER STUDENT BUDGET"].min())


$655.00
$578.00


In [177]:
#create bins   Signature: pd.cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False)

bins = [0,585, 615, 645, 675]

group_labels = ["$0-585", "$585-615", "$615-645", "$645-675"]

#slice data series and place into bins


per_school_summary_df["SPENDING RANGE (PER STUDENT)"] = pd.cut(school_budget, bins, labels = group_labels)

#pd.cut(per_school_summary_df["AVERAGE MATH SCORE"],bins,labels=group_labels)


In [178]:
#GROUP BY SPENDING

KeyError: 'AVERAGE MATH SCORE'

In [182]:
school_merge_df.head(1)

,SCHOOL ID,SCHOOL,SCHOOL TYPE,ALL STUDENTS,BUDGET,STUDENT ID,NAME,GENDER,GRADE,READING SCORE,MATH SCORE,PER STUDENT BUDGET,SPENDING RANGE (PER STUDENT)
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79,<$585,<$585


In [183]:
bins = [0,585, 615, 645, 675]
group_name = ["$0-585", "$585-615", "$615-645", "$645-675"]
school_merge_df['spending_bins'] = pd.cut(school_merge_df['BUDGET']/school_merge_df['ALL STUDENTS'], bins, labels = group_name)

#group by spending
by_spending = school_merge_df.groupby('spending_bins')

#calculations
avg_math = by_spending['MATH SCORE'].mean()
avg_read = by_spending['READING SCORE'].mean()
pass_math = school_merge_df[school_merge_df['MATH SCORE'] >= 70].groupby('spending_bins')['STUDENT ID'].count()/by_spending['STUDENT ID'].count()
pass_read = school_merge_df[school_merge_df['READING SCORE'] >= 70].groupby('spending_bins')['STUDENT ID'].count()/by_spending['STUDENT ID'].count()
overall = school_merge_df[(school_merge_df['READING SCORE'] >= 70) & (school_merge_df['MATH SCORE'] >= 70)].groupby('spending_bins')['STUDENT ID'].count()/by_spending['STUDENT ID'].count()

            
# df build            
scores_by_spend = pd.DataFrame({"Average Math Score": avg_math,"Average Reading Score": avg_read,'% Passing Math': pass_math,
                                '% Passing Reading': pass_read,"Overall Passing Rate": overall})
            
#reorder columns
scores_by_spend = scores_by_spend[["Average Math Score","Average Reading Score",'% Passing Math',
                                   '% Passing Reading',"Overall Passing Rate"]]

scores_by_spend.index.name = "Per Student Budget"
scores_by_spend = scores_by_spend.reindex(group_name)

#formating
scores_by_spend.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Per Student Budget,,,,,
$0-585,83.4,84.0,93.7%,96.7%,90.6%
$585-615,83.5,83.8,94.1%,95.9%,90.1%
$615-645,78.1,81.4,71.4%,83.6%,60.3%
$645-675,77.0,81.0,66.2%,81.1%,53.5%


In [189]:
#GROUP BY SCHOOL SIZE

GRADE,10th,11th,12th,9th
SCHOOL,,,,
Bailey High School,80.91%,80.95%,80.91%,81.30%
Cabrera High School,84.25%,83.79%,84.29%,83.68%
Figueroa High School,81.41%,80.64%,81.38%,81.20%
Ford High School,81.26%,80.40%,80.66%,80.63%
Griffin High School,83.71%,84.29%,84.01%,83.37%
Hernandez High School,80.66%,81.40%,80.86%,80.87%
Holden High School,83.32%,83.82%,84.70%,83.68%
Huang High School,81.51%,81.42%,80.31%,81.29%
Johnson High School,80.77%,80.62%,81.23%,81.26%


In [184]:
# create size bins
bins = [0, 1000, 2000, 5000]
group_name = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_merge_df['size_bins'] = pd.cut(school_merge_df['ALL STUDENTS'], bins, labels = group_name)

#group by spending
by_size = school_merge_df.groupby('size_bins')

#calculations 
avg_math = by_size['MATH SCORE'].mean()
avg_read = by_size['MATH SCORE'].mean()
pass_math = school_merge_df[school_merge_df['MATH SCORE'] >= 70].groupby('size_bins')['STUDENT ID'].count()/by_size['STUDENT ID'].count()
pass_read = school_merge_df[school_merge_df['READING SCORE'] >= 70].groupby('size_bins')['STUDENT ID'].count()/by_size['STUDENT ID'].count()
overall = school_merge_df[(school_merge_df['READING SCORE'] >= 70) & (school_merge_df['MATH SCORE'] >= 70)].groupby('size_bins')['STUDENT ID'].count()/by_size['STUDENT ID'].count()

            
# df build            
scores_by_size = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
            
})
            
#reorder columns
scores_by_size = scores_by_size[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate"
]]

scores_by_size.index.name = "Total Students"
scores_by_size = scores_by_size.reindex(group_name)

#formating
scores_by_size.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Total Students,,,,,
Small (<1000),83.8,83.8,94.0%,96.0%,90.1%
Medium (1000-2000),83.4,83.4,93.6%,96.8%,90.6%
Large (2000-5000),77.5,77.5,68.7%,82.1%,56.6%


In [ ]:
#GROUP BY SCHOOL TYPE

In [188]:
# group by type of school
by_type = school_merge_df.groupby("SCHOOL TYPE")

#calculations 
avg_math = by_type['MATH SCORE'].mean()
avg_read = by_type['MATH SCORE'].mean()
pass_math = school_merge_df[school_merge_df['MATH SCORE'] >= 70].groupby('SCHOOL TYPE')['STUDENT ID'].count()/by_type['STUDENT ID'].count()
pass_read = school_merge_df[school_merge_df['READING SCORE'] >= 70].groupby('SCHOOL TYPE')['STUDENT ID'].count()/by_type['STUDENT ID'].count()
overall = school_merge_df[(school_merge_df['READING SCORE'] >= 70) & (school_merge_df['MATH SCORE'] >= 70)].groupby('SCHOOL TYPE')['STUDENT ID'].count()/by_type['STUDENT ID'].count()

# df build            
scores_by_type = pd.DataFrame({"Average Math Score": avg_math,"Average Reading Score": avg_read,'% Passing Math': pass_math,
                               '% Passing Reading': pass_read,"Overall Passing Rate": overall})
    
#reorder columns
scores_by_type = scores_by_type[["Average Math Score","Average Reading Score",'% Passing Math','% Passing Reading',
                                 "Overall Passing Rate"]]

scores_by_type.index.name = "Type of School"


#formating
scores_by_type.style.format({'Average Math Score': '{:.1f}', 
                              'Average Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Type of School,,,,,
Charter,83.4,83.4,93.7%,96.6%,90.6%
District,77.0,77.0,66.5%,80.9%,53.7%
